<a href="https://colab.research.google.com/github/trehansalil/amazon_sales_forecasting/blob/main/Amazon_sales_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'unlock-profits-with-e-commerce-sales-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2699295%2F4642886%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240417%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240417T062348Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D30bde75265db234df1e4cb15dbb2c252ea8e5e350c094011aec3007975cb22dada7ebb0f98bf51227bf960c2afca70944bf8229a7adbcd8095a924485d25d8b9bee118a11935a6da2127eda2ffed265dc87fda65023d3bb07e37b0526abb56fb0a4608bf2e59c52721bf1c69e36d2357d1d1736517747bf0e9bf1d07fe244e8466e4fe989fc86a091ec92b31950c95aa222aed29fee5f015b62ed796c3ebbe404115cf4b5fc8c9e1529ba3d6bbfa764a46a918aced4d1943d72ee9b3c965267b183bf3942ffdea41a80ec27057c73518a5657f0332c911bc4f679272fb172eec44e8eeda6d631b4908232dadb2a9a3ce511c2abb559ad85317913ea2bcc8c7be'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
import numpy as np
import pandas as pd
random.seed(42)
alpha = 0.05
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import catboost
import xgboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

## Data Dictionary
This dataset provides detailed insights into Amazon sales data, including SKU Code, Design Number, Stock, Category, Size and Color, to help optimize product profitability

- Category: Type of product. (String)
- Size: Size of the product. (String)
- Date: Date of the sale. (Date)
- Status: Status of the sale. (String)
- Fulfilment: Method of fulfilment. (String)
- Style: Style of the product. (String)
- SKU: Stock Keeping Unit. (String)
- ASIN: Amazon Standard Identification Number. (String)
- Courier Status: Status of the courier. (String)
- Qty: Quantity of the product. (Integer)
- Amount: Amount of the sale. (Float)
- B2B: Business to business sale. (Boolean)
- Currency: The currency used for the sale. (String)

In [ ]:
amazon_df = pd.read_csv('/kaggle/input/unlock-profits-with-e-commerce-sales-data/Amazon Sale Report.csv', low_memory=False)
amazon_df = amazon_df.drop(columns = ['index', 'Unnamed: 22']).reset_index(drop=True)

# Map sizes to aggregated sizes
size_map = {'S': 'S',
 '3XL': '3XL',
 'XL': 'XL',
 'L': 'L',
 'XXL': '2XL',
 'XS': 'XS',
 '6XL': '6XL',
 'M': 'M',
 '4XL': '4XL',
 '5XL': '5XL',
 'Free': 'FREE'}

# size_map['Free'] = 'FREE'

amazon_df["ship_region"] = amazon_df["ship-state"].str.upper().replace({
        "MAHARASHTRA":"westindia","KARNATAKA":"southindia",
        'PUDUCHERRY':"southindia",'TELANGANA':"southindia",
        'ANDHRA PRADESH':"southindia", 'HARYANA':"northindia",
        'JHARKHAND':"eastindia", 'CHHATTISGARH':"eastindia",
        'ASSAM':"northeastindia",'ODISHA':"eastindia", "ORISSA": "eastindia",
        'UTTAR PRADESH':"northindia", 'GUJARAT':"westindia",
        'TAMIL NADU':"southindia", 'UTTARAKHAND':"northindia",
        'WEST BENGAL':"eastindia", 'RAJASTHAN':"westindia", 'RAJSHTHAN':"westindia", 'RJ':"westindia", 'RAJSTHAN':"westindia",
        'NEW DELHI':"centralindia",'MADHYA PRADESH':"centralindia", "NL": 'northeastindia', 'AR': 'northeastindia',
        'KERALA':"southindia", 'JAMMU & KASHMIR':"northindia",
        'BIHAR':"eastindia",'MEGHALAYA':"northeastindia",
        'PUNJAB':"northindia", 'PB': 'northindia', 'GOA':"southindia", 'PUNJAB/MOHALI/ZIRAKPUR': "northindia",
        'TRIPURA':"northeastindia", 'CHANDIGARH':"northindia",
        'HIMACHAL PRADESH':"northindia",'SIKKIM':"northeastindia",
        "ANDAMAN & NICOBAR ":"eastindia", 'MANIPUR':"northeastindia",
        'MIZORAM':"northeastindia",'NAGALAND':"northeastindia",
        'ARUNACHAL PRADESH':"northeastindia", 'LADAKH':"northindia", "PONDICHERRY": "westindia",
        'DADRA AND NAGAR':"westindia",'LAKSHADWEEP':"southindia", 'DELHI': 'northindia'
    })

values = {"Size": 'FREE'}
amazon_df.fillna(value=values, inplace=True)

amazon_df['Size'] = amazon_df['Size'].map(size_map)
amazon_df['AggSize'] = amazon_df['Size'].apply(lambda x: '>2XL' if ('XL'in x) & (x not in ['XL', '2XL']) else x)

amazon_df['Date'] = pd.to_datetime(amazon_df['Date'], dayfirst=True, format='mixed')

amazon_df['B2B'] = amazon_df['B2B'].astype(int)


In [ ]:
amazon_df["ship_region"].unique()

In [ ]:
amazon_df['Size'].unique()

In [ ]:
amazon_df['Size'].value_counts(1)

In [ ]:
amazon_df['AggSize'].value_counts(1)

In [ ]:
round(amazon_df['Size'].value_counts(1)*100, 2)

In [ ]:
amazon_df['Size'].isnull().sum()

In [ ]:
amazon_df.columns

In [ ]:
amazon_df.loc[amazon_df['Size'].isin(['FREE']), 'SKU'].apply(lambda x: x.split("-")[-1]).unique()

In [ ]:
amazon_df.groupby('Status')['Courier Status'].unique()

In [ ]:
pd.set_option("display.max_columns", 101)
amazon_df.head()

In [ ]:
amazon_df.info()

In [ ]:
amazon_df.dtypes

In [ ]:
amazon_df.nunique().sort_values().to_frame(name='Count of unique values')

In [ ]:
amazon_df.apply(pd.unique).to_frame(name='Unique Values')

In [ ]:
amazon_df.describe().T

In [ ]:
amazon_df.describe(include='O')

In [ ]:
sns.heatmap(amazon_df.isnull())
plt.show()

# Null values are there in Courier Status, Currency, Amount, promotion-ids, fulfilled-by

In [ ]:
def null_checker():
    return round(amazon_df.isnull().sum()[amazon_df.isnull().sum()>0]* 100/amazon_df.shape[0], 2)
null_checker()

### Drop Duplicates

In [ ]:
amazon_df[amazon_df.duplicated(['Order ID','ASIN'], keep=False)]

In [ ]:
len(amazon_df)-len(amazon_df.drop_duplicates(['Order ID','ASIN']))

In [ ]:
amazon_df.drop_duplicates(['Order ID','ASIN'],inplace = True,ignore_index=True)

### Filling NaN Values

In [ ]:
amazon_df['currency'].unique()

In [ ]:

amazon_df['ship-country'].unique()

In [ ]:
amazon_df.columns

In [ ]:
amazon_df.groupby('currency')['ship-country'].unique()

In [ ]:
amazon_df.drop(columns = ['currency', 'ship-country'], inplace=True)

# It is safe to assume that these are local sales from Amazon. Hence, there is no reason to consider these columns as the values aren't going to change much

In [ ]:
amazon_df.columns

In [ ]:


amazon_df.groupby('Fulfilment')['fulfilled-by'].unique()

# All Amazon Fulfillments are not specified in this data set. Lets assume Amazon's Shipment was a different category of Fulfillment - by.

# Its safe to drop fullfilment-by as there is no separate information brought to the table through fulfilled-by

In [ ]:
amazon_df.drop(columns=['fulfilled-by'], inplace=True)

In [ ]:
amazon_df['Sales Channel '].unique()

In [ ]:
amazon_df[amazon_df['Amount'].isnull()]['Status'].value_counts(normalize=True).apply(lambda x: format(x, '.2%'))

In [ ]:
values = {'Amount': 0}
amazon_df.fillna(values,inplace=True)

In [ ]:
amazon_df.groupby('Fulfilment')['Sales Channel '].unique()

In [ ]:
null_checker()

In [ ]:
amazon_df.columns

In [ ]:
amazon_df['Combined'] = amazon_df['Order ID'].astype(str) + '-' + amazon_df['SKU'].astype(str)

In [ ]:
courier_nulls_df = amazon_df.loc[amazon_df['Courier Status'].isnull(), :].groupby(['Order ID', 'SKU'])['Courier Status'].unique().reset_index()
courier_nulls_df

In [ ]:
amazon_df.loc[(~amazon_df['Courier Status'].isnull()) & (amazon_df['Order ID'].isin(courier_nulls_df['Order ID'].unique())), :]

#

In [ ]:
# lets assume Status of the order to reflect the most common Courier Status across it to fill up the null values
status_courier_stat_dict = amazon_df.groupby('Status')['Courier Status'].apply(lambda x: x.mode()[0] if not x.empty else None).to_dict()

# for status in status_courier_stat_dict:

In [ ]:
amazon_df.loc[amazon_df['Courier Status'].isnull(),
              "Courier Status"] = amazon_df.loc[amazon_df['Courier Status'].isnull(), "Status"].apply(lambda x: status_courier_stat_dict[x])

In [ ]:
null_checker()

In [ ]:
ship_postal_code_nulls_df = amazon_df.loc[amazon_df['ship-postal-code'].isnull(), :].groupby(['Combined'])['ship-postal-code'].unique().reset_index()
ship_postal_code_nulls_df

In [ ]:
amazon_df.loc[(~amazon_df['ship-postal-code'].isnull()) & (amazon_df['Combined'].isin(ship_postal_code_nulls_df['Combined'].unique())), :]

# Hence, there is no way to identify ship-postal-code/ship-city/ship-state. Hence, its safe to drop the rows with nulls in these columns. Anyway, the valume of nulls is very few so it shouldn't affect the analysis too much.



In [ ]:
amazon_df.drop(index=amazon_df.loc[(amazon_df['ship-postal-code'].isnull()), :].index, inplace=True)

In [ ]:
null_checker()

In [ ]:
amazon_df['promotion-ids'].nunique()

In [ ]:
amazon_df['Combined'].nunique()

In [ ]:
amazon_df['promotion-ids'][0] is np.nan

In [ ]:
def condition_checker(x):
    if x is np.nan:
        return x
    else:
        return x.split(",")

In [ ]:
amazon_df['promotion-id_'] = amazon_df['promotion-ids'].apply(lambda x: condition_checker(x=x))

In [ ]:
amazon_df['promotion-id_'].explode().nunique()

In [ ]:
values = {'promotion-ids': "No Promotion"}
amazon_df.fillna(values,inplace=True)

# Assume some Other class here

In [ ]:
amazon_df['_is_promotion_free_financing'] = amazon_df['promotion-ids'].str.contains('Amazon PLCC Free-Financing Universal Merchant').astype(int)
amazon_df['_is_core_free_ship'] = amazon_df['promotion-ids'].str.contains('Core Free Shipping').astype(int)
amazon_df['_is_promotion_coupon'] = amazon_df['promotion-ids'].str.contains('Coupon').astype(int)

In [ ]:
amazon_df['_is_promotion_free_fin_coup_core_free_ship'] = amazon_df['_is_promotion_free_financing'] + amazon_df['_is_promotion_coupon'] + amazon_df['_is_core_free_ship']

In [ ]:
amazon_df = amazon_df.reset_index(drop=True).drop(columns=['promotion-id_', 'promotion-ids'])

In [ ]:
null_checker()

### Analyzing Discrepancies

In [ ]:
amazon_df.loc[(amazon_df['Qty']==0)
              & (~amazon_df['Status'].isin(['Cancelled', 'Pending', 'Shipped - Returned to Seller'])
                 & (~amazon_df['Courier Status'].isin(['Cancelled']))) , :]

# Understanding the discrepancies w.r.t Qty variable. These orders have different problems:
# The crux of all is that Qty == 0 are not relevant for forecastinf exercise. Hence removing all rows with Qty ==0.

In [ ]:
amazon_df.loc[amazon_df['Amount']==0]
# These must be replacements, even though Quantity of product is shipped, there is no revenue from it. Hence removing these as well.

In [ ]:
amazon_df.loc[(amazon_df['Qty']==0) | (amazon_df['Amount']==0)].index

In [ ]:
amazon_df.shape[0]

In [ ]:
amazon_df.drop(index=amazon_df.loc[(amazon_df['Qty']==0) | (amazon_df['Amount']==0)].index, inplace=True)

amazon_df.reset_index(drop=True, inplace=True)
amazon_df.drop(columns=['ship-city', 'ship-state', 'ship-postal-code'], inplace=True)

In [ ]:
amazon_df.info()

### Feature Engineering


In [ ]:


# Extracting date features

amazon_df['month'] = amazon_df['Date'].dt.month
# amazon_df['day'] = amazon_df['Date'].dt.day
amazon_df['day_of_week'] = amazon_df['Date'].dt.dayofweek  # Monday=0, Sunday=6
# amazon_df['day_of_year'] = amazon_df['Date'].dt.dayofyear
# amazon_df['week_of_year'] = amazon_df['Date'].dt.isocalendar().week
amazon_df['quarter'] = amazon_df['Date'].dt.quarter

# Additional features
amazon_df['is_weekend'] = amazon_df['Date'].dt.dayofweek // 5  # 1 if weekend, 0 if weekday

In [ ]:
amazon_df.nunique().sort_values().to_frame(name='Count of unique values')

In [ ]:
amazon_df.drop(columns=['Sales Channel ', 'Combined'],inplace=True)

In [ ]:
amazon_df['amt_per_item'] = amazon_df['Amount']/amazon_df['Qty']

In [ ]:
amazon_df.columns

In [ ]:
amazon_df

In [ ]:
for i in ['_is_promotion_free_financing',
       '_is_core_free_ship', '_is_promotion_coupon', 'is_weekend', 'B2B', '_is_promotion_free_fin_coup_core_free_ship']:
    amazon_df[i] = amazon_df[i].astype('category')

In [ ]:
target_col = 'Amount'
cat_cols = amazon_df.select_dtypes(include=['object', 'category']).columns
num_cols = amazon_df.drop(columns=[target_col]).select_dtypes(include=['int', "float"]).columns



print(cat_cols)
print(num_cols)

### Univariate Analysis


In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
data = amazon_df
for num_col in num_cols:
    stat, p_value = stats.shapiro(data[num_col])
    if p_value > alpha:
        print(f'{num_col} looks Gaussian (fail to reject H0)')
    else:
        print(f'{num_col} doesnt looks Gaussian')
    sns.distplot(data[num_col])
    plt.show()
# Hence, neither of the distributions are Gaussian at 95% ci.

### Bivariate Analysis

In [ ]:
relavant_variables = []
for num_col in num_cols:
    stat, p_value = stats.pearsonr(data[num_col],data[target_col])
    if p_value > alpha:
        print(f'{num_col} means both the samples are independent (fail to reject H0)')
    else:
        print(f'{num_col} means there is a dependency between the samples with p_value:{p_value} (reject the H0)')
        relavant_variables.append(num_col)
        sns.scatterplot(data=data, x=target_col, y=num_col)
        plt.show()